# TensorFlow 原理

【参考】
- [cnblogs - TensorFlow核心概念和原理介绍](https://www.cnblogs.com/wkslearner/archive/2018/01/03/8185890.html)

TensorFlow 是一个采用**数据流图**（data flow graphs），用于**数值计算**的开源软件库。**节点**（Nodes）在图中表示数学操作，图中的**线**（edges）则表示在节点间相互联系的多维数据数组，即张量（tensor）。

## 数据流图（data flow graph）
数据流图用“结点”（nodes）和“线”(edges)的有向图来描述数学计算。

- “节点” 一般用来表示施加的数学操作，但也可以表示数据输入（feed in）的起点/输出（push out）的终点，或者是读取/写入持久变量（persistent variable）的终点。
- “线”表示“节点”之间的输入/输出关系。这些数据“线”可以输运“size可动态调整”的多维数据数组，即“张量”（tensor）。

张量从图中流过的直观图像是这个工具取名为“Tensorflow”的原因。一旦输入端的所有张量准备好，节点将被分配到各种计算设备完成异步并行地执行运算：
![img](https://ws4.sinaimg.cn/large/69d4185bly1fw8wckhndyg20700cgwol.gif)

## 计算图
【参考】
- [csdn - tensorflow中有向图（计算图、Graph）、上下文环境（Session）和执行流程](https://blog.csdn.net/dcrmg/article/details/79028003)

Tensorflow是基于图(Graph)的计算框架，图的节点由事先定义的运算(操作、Operation)构成，图的各个节点之间由张量（tensor）来链接，Tensorflow的计算过程就是张量（tensor）在节点之间从前到后的流动传输过程，如下图示例:
![img](https://ws2.sinaimg.cn/large/69d4185bly1fw8wm6spicj209708l74v.jpg)

有向图中，节点通常代表数学运算，边表示节点之间的某种联系，它负责传输多维数据(Tensors)。

节点可以被分配到多个计算设备上，可以异步和并行地执行操作。因为是有向图，所以只有等到之前的入度节点们的计算状态完成后，其后的节点才能执行操作。**推广到神经网络中，同一层之间的不同节点上的运算可以异步或并行的执行，但是前后层之间的执行还是要顺序执行，因为后一层的输入依赖于前一层的输出**。


在TensorFlow中，首先需要构建一个计算图，然后按照计算图启动一个会话，*在会话中完成变量赋值，计算，得到最终结果等操作*。因此，可以说TensorFlow是一个按照计算图设计的逻辑进行计算的编程系统。

TensorFlow的计算图可以分为两个部分：
- 构造部分，包含计算流图；
- 执行部分，通过session执行图中的计算。

构造部分又分为两部分：
- 创建源节点；
- 源节点输出传递给其他节点做运算。

TensorFlow默认图：TensorFlow python库中有一个默认图(default graph)。节点构造器(op构造器)可以增加节点。

## 张量（tensor）
【参考】
- [简书 - TensorFlow之计算图、张量和会话](https://www.jianshu.com/p/219908f652f5)

张量，可以被简单地理解为多维数组，

在TF程序中，所有的数据都通过张量（tensor）的形式来表示，与 Caffe 中的 blob 相似，张量可以简单的理解为**多维数组**。零阶张量表示标量（scalar）也就是一个数；第一阶张量为向量（vector）也就是一维数组；第n阶张量可以理解为一个n维数组。

与numpy中数组不同的是张量还包含三个重要属性名字、维度、类型。张量的名字，是张量的**唯一标识符**，通过名字可以发现张量是如何计算出来的，$\color{red}{张量其实并没有真正保存数字，它保存的是如何得到这些数字的计算过程}$。比如`add:0`代表的是计算节点`add`的第一个输出结果：
```python
a = tf.constant([1,2], name="a")
b = tf.constant([3,4], name="b")
result = tf.add(a, b)
print(result)
Tensor("Add:0", shape=(2,), dtype=int32)
```
从上面的代码可以看出TF中的张量和Numpy中数组的不同，TF计算的结果保存了三个属性：名字（name）、维度（shape）、类型（type）。

维度和类型与数组类似。



## 会话（session）
`Session`对象将操作节点`op`封装在一定的环境内运行，同时`tensor`对象将被计算求值。 一个`Session`可能会拥有一些资源，例如`Variable`或者`Queue`。当我们不再需要该`session`的时候，需要调用`sess.close()`关闭会话(或使用Python上下文管理器自动关闭)，将这些资源释放。

如果在创建`Session`时没有指定`Graph`，则该`Session`会加载默认`Graph`。如果在一个进程中创建了多个`Graph`，则需要创建不同的`Session`来加载每个`Graph`，而每个`Graph`则可以加载在多个`Session`中进行计算，`Session`与`Session`之间是相互独立的。

在TensorFlow中使用会话有两种方式。第一种需要明确调用会话生成函数和关闭会话函数:
```python
# 创建session  
session = tf.Session()  
#获取运算结果  
session.run()  
#关闭会话,释放资源  
session.close()
```

第二种可以使用with的方式:
```python
with tf.Session() as session:  
    session.run()
```
两种方式不同的是，第二种限制了 `session` 的作用域，即 `session` 这个参数只适用于 `with` 语句下面，同时语句结束后自动释放资源，而第一种方式 `session` 则作用于整个程序文件，需要用 `close` 来释放资源。

## 执行流程
Tensorflow的执行可以分为三个步骤：
- 创建Tensor
- 定义Operations（Operations输入Tensor，然后输出另一个Tensor），即如何操作 tensor
- 执行计算（也就是运行一个可计算的图）

一个最简单的Tensorflow计算流程示例如下，实现两个一维向量求和运算：
```python 
# 1. 创建两个张量（Tensor），定义了具体的数据
input1 = tf.constant([1.0, 1.0, 1.0, 1.0])
input2 = tf.constant([2.0, 2.0, 2.0, 2.0])
 
# 2. 定义操作（Operations），如何操作 tensor
output = tf.add(input1, input2)
 
# 3.  执行计算
with tf.Session() as sess:
    result = sess.run(output)
    #result = output.eval()
    print result
```

tf中是定义操作（op）与执行操作分开的工作模式。上例中定义了一个名称为output的求和操作(也就是图(Graph)上的一个节点)，定义操作(op)的时候计算不会立即执行，直到在上下文的Session环境中调用`run()`或`eval()`，触发了Tensorflow执行计算图，从而获取output计算结点的结果。

更一般的情况下，$\color{red}{tf中输入数据可能是不确定或随时变化的}$，针对这种情况，tf提供了 `feed` 注入机制，可以临时替代图（Graph）中操作(op)的输入张量（tensor），即把这些数据定义为**占位变量**，直到执行计算图之前才具体给定数据，再把这些数据以`feed_dict`的形式作为参数提供给`sess.run()`调用。feed机制按这种思路，上例改为：
```python
# 1. 创建两个占位变量，只定义数值类型和形状（shape），具体数值在计算图执行前给定
input1 = tf.placeholder(tf.float16,shape=[4])
input2 = tf.placeholder(tf.float16,shape=[4])
 
# 2. 定义操作（Operations）
output = tf.add(input1, input2)
 
# 3.  执行计算
with tf.Session() as sess:
    input_1 = [1.0, 1.0, 1.0, 1.0]
    input_2 = [2.0, 2.0, 2.0, 2.0]
 
    result = sess.run(output, feed_dict = {input1:input_1, input2:input_2})
    print result
```

## 分布式
tensorflow的实现分为了单机实现和分布式实现。

单机的模式下，计算图会按照程序间的依赖关系顺序执行。

在分布式实现中，需要实现的是对client，master，worker process，device管理。

- client也就是客户端，他通过session的接口与master和worker相连。
- master则负责管理所有woker的计算图执行。
- worker由一个或多个计算设备device组成，如cpu，gpu等。

具体过程如下图：
![img](https://ws2.sinaimg.cn/large/69d4185bly1fw8z2h413sj20yg0bjwiu.jpg)

在分布式实现中，tensorflow有一套专门的节点分配策略。策略是基于**代价模型**，代价模型会估算每个节点的输入，输出的tensor大小以及所需的计算时间，然后分配每个节点的计算设备。

## 扩展功能
在tensorflow中比较重要的拓展功能有，自动求导，子图执行，计算图控制流以及队列/容器

- 自动求导：求导是机器学习中计算损失函数常用的运算，TensorFlow原生支持自动求导运算，它是通过计算图中的拓展节点实现。
- 子图执行：是通过控制张量的流向实现。
- 计算图控制流：是指控制计算图的节点极其运行的设备管理，它提供了快速执行计算和满足设备施加的各种约束。比如限制内存总量为了执行它的图子集而在设备上所需的节点。
- 队列：是一个有用的功能，它们允许图的不同部分异步执行，对数据进行入队和出队操作。
- 容器：是用来存放变量，默认的容器是持久的，直到进程终止才会清空，同时容器中的变量也可以共享给其他计算图使用。